In [1]:
import os
import torch
import random
import datasets

import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from datetime import datetime
from transformers import AdamW
from transformers import get_scheduler
from torch.utils.data import DataLoader
from IPython.display import display, HTML
from datasets import Dataset, DatasetDict, load_dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [2]:
def process_data():
    """
    Output: Translated text of the source text
    :param src_text: source string
    """ 
    
    #Load data (needs to be changed to loading from BQ or bucket)
    df = pd.read_csv('../../data/en_GB-nb_NO/en_GB-nb_NO_batch_0.csv', engine='python')

    df = df[['source', 'target']] # will be redundent when setup 
    df['source'] = df['source'].apply(lambda s: str(s))
    df['target'] = df['target'].apply(lambda s: str(s))

    # Transform to HuggingFace Dataset
    data = Dataset.from_pandas(pd.DataFrame({'translation': df.to_dict('records')})) 

    # Split data into training sets
    train_test_valid = data.train_test_split(shuffle=True, seed=7, test_size=0.0015)
    test_valid = train_test_valid['test'].train_test_split(shuffle=True, seed=7, test_size=0.5)

    # Convert to train/validate/test
    dataset = DatasetDict({
        'train': train_test_valid['train'],
        'validation': test_valid['test'],
        'test': test_valid['train']})
     
    return dataset

In [3]:
def tokenization_processing(dataset, source_len=128, target_len=128,
                            source="source", target="target"):
    """
    Output: Generates tokenized data using the attributes of the base model
    :param dataset: raw dataset to transform
    :param source_len: maximum sentence length for source string
    :param target_len: maximum sentence length for target string
    :param source: source language
    :param target: target language
    """
    
    inputs = [s[source] for s in dataset["translation"]]
    targets = [s[target] for s in dataset["translation"]]
    
    model_inputs = tokenizer(inputs, max_length=source_len, truncation=True)
    
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=target_len, truncation=True)
        
    model_inputs["labels"] = labels["input_ids"]
    
    return model_inputs

In [4]:
def compute_metrics(predictions): # n.b. can add more metrics later
    """
    Output: evaluation metrics to track model performance in training
    :param predictions: output of predictions to decode
    """
    
    metric = load_metric("sacrebleu")
    
    def process_text(predictions, labels):
        preds = [pred.strip() for pred in predictions]
        labels = [[label.strip()] for label in labels]
        return preds, labels
    
    preds, labels = predictions
    
    if isinstance(preds, tuple):
        preds = preds[0]
    
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = process_text(decoded_preds, decoded_labels)
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    
    result = {k: round(v, 6) for k, v in result.items()} #round results
    
    return result

In [6]:
model_name="Helsinki-NLP/opus-mt-en-no"

dataset = process_data()

print('load models and data collator')
trained_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=trained_model)

load models and data collator


OSError: We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like Helsinki-NLP/opus-mt-en-no is not the path to a directory containing a config.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [6]:
tokenized_dataset = dataset.map(tokenization_processing, batched=True)

  0%|          | 0/200 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
time = str(datetime.now())

batch_size=16
learning_rate=2e-5 
weight_decay=0.01
save_limit=10
epochs=20

# n.b. need to add early stopping with longer training times
args = Seq2SeqTrainingArguments(
    f"../../models/checkpoints/IKEA_MT_en_GB-it_IT_{time}",
    evaluation_strategy = "epoch",
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay = weight_decay,
    save_total_limit = save_limit,
    num_train_epochs = epochs,
    predict_with_generate=True    
)


trainer = Seq2SeqTrainer(
    trained_model,
    args,
    train_dataset = tokenized_dataset["train"],
    eval_dataset = tokenized_dataset["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)


In [ ]:
print('train model')
trainer.train()

The following columns in the training set don't have a corresponding argument in `MarianMTModel.forward` and have been ignored: translation. If translation are not expected by `MarianMTModel.forward`,  you can safely ignore this message.
/home/jupyter/.virtualenvs/ikea-mt/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 199112
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 248900


train model


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ../../models/checkpoints/IKEA_MT_en_GB-it_IT_2022-12-23 14:07:24.526177/checkpoint-500
Configuration saved in ../../models/checkpoints/IKEA_MT_en_GB-it_IT_2022-12-23 14:07:24.526177/checkpoint-500/config.json
Model weights saved in ../../models/checkpoints/IKEA_MT_en_GB-it_IT_2022-12-23 14:07:24.526177/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ../../models/checkpoints/IKEA_MT_en_GB-it_IT_2022-12-23 14:07:24.526177/checkpoint-500/tokenizer_config.json
Special tokens file saved in ../../models/checkpoints/IKEA_MT_en_GB-it_IT_2022-12-23 14:07:24.526177/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ../../models/checkpoints/IKEA_MT_en_GB-it_IT_2022-12-23 14:07:24.526177/checkpoint-1000
Configuration saved in ../../models/checkpoints/IKEA_MT_en_GB-it_IT_2022-12-23 14:07:24.526177/checkpoint-1000/config.json
Model weights saved in ../../models/checkpoints/IKEA_MT_en_GB-it_IT_2022-12-23 14:07:24.526177/checkpoint-1000/pytorch

In [ ]:
print('save model')
trainer.save_model('../../models/en_GB-it_IT/IKEA_MT-en_it_IT_' + time)